We’ll use Natural Language Processing (NLP) to automatically generate audit reports based on detected risks & anomalies.

In [1]:
!pip install transformers

In [3]:
from transformers import pipeline
import pandas as pd

In [4]:
# Load high-risk tools data
df_high_risk = pd.read_csv("High_Risk_Tools.csv")

In [5]:
# Load Pretrained Text Generation Model
report_generator = pipeline("text-generation", model="gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [6]:
# Check if there are high-risk tools
if df_high_risk.empty:
    print("\n No high-risk tools found! Adjust risk threshold if needed.")
else:
    print(f"\n Loaded {len(df_high_risk)} High-Risk Tools for Report Generation!")


 Loaded 4 High-Risk Tools for Report Generation!


In [7]:
# Function to Generate Reports
def generate_report(tool_name, risk_score, days_since_last_cal, months_remaining, anomaly_flag):
    prompt = f"""
    Compliance Report for {tool_name}:
    - Risk Score: {risk_score:.2f}/100
    - Days Since Last Calibration: {days_since_last_cal}
    - Months Remaining Before Next Calibration: {months_remaining}
    - Anomaly Detected: {"Yes" if anomaly_flag else "No"}

    Based on this data, the compliance risk assessment indicates:
    """

    # Generate Report
    report = report_generator(prompt, max_length=200, num_return_sequences=1)[0]["generated_text"]
    return report

In [9]:
# Generate Reports for High-Risk Tools
df_high_risk = df_high_risk[df_high_risk["Risk_Score"] > 70]  # Tools with high risk
df_high_risk["Compliance_Report"] = df_high_risk.apply(
    lambda row: generate_report(row["Tool_Description"], row["Risk_Score"],
                                row["Days_Since_Last_Calibration"], row["Months_Remaining"],
                                row["Anomaly_Score_AE"]),
    axis=1
)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
# Save Compliance Reports
df_high_risk[["Tool_Description", "Risk_Score", "Compliance_Report"]].to_csv("Compliance_Reports.csv", index=False)

In [11]:
# Display Sample Reports
df_high_risk[["Tool_Description", "Risk_Score", "Compliance_Report"]].head(5)

,Tool_Description,Risk_Score,Compliance_Report
0,Tachometer (non-contact),74.271614,\n Compliance Report for Tachometer (non-co...


In [13]:
# Print Full Reports for High-Risk Tools

df_reports = pd.read_csv("Compliance_Reports.csv")

for index, row in df_reports.iterrows():
    print(f"\n🔹 Compliance Report for {row['Tool_Description']}:")
    print(f"Risk Score: {row['Risk_Score']:.2f}/100")
    print(f"Report:\n{row['Compliance_Report']}")
    print("="*80)  # Separator for readability


🔹 Compliance Report for Tachometer (non-contact):
Risk Score: 74.27/100
Report:

    Compliance Report for Tachometer (non-contact):
    - Risk Score: 74.27/100
    - Days Since Last Calibration: 0.6344892221180881
    - Months Remaining Before Next Calibration: 0.3702651515151515
    - Anomaly Detected: Yes

    Based on this data, the compliance risk assessment indicates:
     1.15. This risk may occur within 15 days of the last calibration event when the calibration may not be performed on any later measurement or because of an error in the measurement or calibration of the instrument. If the instrument is performing the calibration correctly, if the calibration is not properly performed on any further calibration, the calibration may result in an accident at the calibration location or the instrument may fail to calibrate. 2. The risk is greater if the instrument's operating control will not be


In [14]:
print("\n AI-Generated Compliance Reports Saved!")


 AI-Generated Compliance Reports Saved!
